<a href="https://colab.research.google.com/github/protovici/PyTorch_Lightning_Dev/blob/main/lightning_cnn_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a CNN model for image recognition | Page 38

In [1]:
!pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
!pip install pytorch-lightning==1.5.2 --quiet
!pip install seaborn==0.11.2
!pip install numpy
!pip install pandas==1.3.5
!pip install opendatasets --upgrade --quiet

     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:38tcmalloc: large alloc 1147494400 bytes == 0x663f6000 @  0x7fb19e7d2615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 881.9 MB 18 kB/s 
     |████████████████████████████████| 23.3 MB 13.1 MB/s 
     |████████████████████████████████| 8.0 MB 4.2 MB/s 
     |████████████████████████████████| 2.9 MB 37.7 MB/s 
     |████████████████████████████████| 1.0 MB 9.1 MB/s 
     |████████████████████████████████| 529 kB 37.1 MB/s 
     |████████████████████████████████| 829 kB 53.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pu

In [2]:
import os
import shutil
import opendatasets as od
import random
import types
import pkg_resources
import gc
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchmetrics.functional import accuracy
import pytorch_lightning as pl

# Set random seed and random seed for all lightning models:
RANDOM_SEED=42
pl.seed_everything(RANDOM_SEED)

# Processing Device(s):
processing_device= "cuda" if torch.cuda.is_available() else "cpu"
torch_aval = torch.cuda.is_available()

# Labels:
lable_1 = 'ENVIRONMENT SET-UP WITH DESCRIPTIONS'
lable_2 = 'If NVIDIA-SMI is not found, then CUDA isnt available on this device:'
lable_3 = 'ENVIRONMENT IS COMPLETE & STAGED CORRECTLY'

def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Set data paths:
IN=r'/content/data/in'
OUT=r'/content/data/out'

print(f'{lable_1:-^80}')
print()
print(f"Location of inbound data: ", IN)
print(f"Location of outbound data: ", OUT)
print(f"Versions: ", requirements)
print(f"Device(s) to Utilize: ", processing_device)
print(f"Is Torch GPU Avaliable?: ", torch_aval)
print(f"# of Devices Found: ", torch.__version__)
print("Torch version:",torch.__version__)
print("Pytorch ligthening version:",pl.__version__)
print()
print(f'{lable_2:-^80}')
!nvidia-smi
gc.collect()
print()
print(f"Clear Torch Cuda Memory: ", torch.cuda.empty_cache())
print(f'{lable_3:-^80}')

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


----------------------ENVIRONMENT SET-UP WITH DESCRIPTIONS----------------------

Location of inbound data:  /content/data/in
Location of outbound data:  /content/data/out
Versions:  [('torchvision', '0.13.1+cu113'), ('torch', '1.12.1+cu113'), ('seaborn', '0.11.2'), ('Pillow', '7.1.2'), ('pandas', '1.3.5'), ('numpy', '1.21.6'), ('matplotlib', '3.2.2')]
Device(s) to Utilize:  cpu
Is Torch GPU Avaliable?:  False
# of Devices Found:  1.10.0+cu102
Torch version: 1.10.0+cu102
Pytorch ligthening version: 1.5.2

------If NVIDIA-SMI is not found, then CUDA isnt available on this device:------
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Clear Torch Cuda Memory:  None
-------------------ENVIRONMENT IS COMPLETE & STAGED CORRECTLY-------------------
